In [1]:
import pickle
import os

os.chdir("/Users/noideaatall/_trash")

In [10]:
import pandas as pd
import numpy as np

submissions = pd.read_pickle("submissions.pkl", compression=None)

In [5]:
submissions.dtypes

airtable_id                  object
amount_paid                 float64
city                         object
course_code                  object
course_id                    object
course_status                 Int64
course_type                   Int64
date_processed       datetime64[ns]
date_received        datetime64[ns]
payment_id                    Int64
payment_link                 object
school_name                  object
school_year                  object
site_order_number            object
student_email                object
student_id                   object
student_name                 object
student_phone                object
student_surname              object
submission_id                object
dtype: object

In [15]:
def format_phone(phone):
    if phone != phone:
        return np.nan
    phone = str(phone)
    if phone.startswith("8") and (phone[1:2] == "9" or phone[1:2] == "4"):
        phone = "+7" + phone[1:]
    else:
        phone = "+" + phone
    return phone

submissions['phone2'] = submissions['student_phone'].apply(format_phone)
submissions.dropna(subset=['student_phone'], inplace=True)

submissions[~submissions['phone2'].str.startswith("+7")].head()

,airtable_id,amount_paid,city,course_code,course_id,course_status,course_type,date_processed,date_received,payment_id,...,school_name,school_year,site_order_number,student_email,student_id,student_name,student_phone,student_surname,submission_id,phone2
145,NaN,7000.0,NaN,VWX_2.23.01,B0557,3,1,NaT,NaT,1,...,NaN,NaN,NaN,illluminii@gmail.com,A0718,Марина,61490942371,Рукавишникова,P00182,+61490942371
382,NaN,10500.0,NaN,RHN_11.10.12,B0627,3,1,NaT,NaT,1,...,NaN,NaN,NaN,mashpanova@gmail.com,A0930,Маша,491732692492,Панова,P00497,+491732692492
735,NaN,11000.0,NaN,PRT_5.1.06,B0678,3,1,NaT,NaT,1,...,NaN,NaN,NaN,plekkatya@yandex.ru,A1197,Катерина,4915901344146,Плеханова,P00800,+4915901344146
785,NaN,12000.0,NaN,CAD_4.20.06,B0685,3,1,NaT,NaT,1,...,NaN,NaN,NaN,plekkatya@yandex.ru,A1197,Катерина,4915901344146,Плеханова,P00869,+4915901344146
991,120,13000.0,NaN,RVT_6.26.09,B0725,3,1,NaT,NaT,1,...,NaN,NaN,NaN,mashpanova@gmail.com,A0930,Маша,491732692492,Панова,P01019,+491732692492


In [51]:
import phonenumbers
from phonenumbers.phonenumberutil import region_code_for_country_code

def phone_check(phone):
    try:
        x = phonenumbers.parse(phone, None)
        possible = "P" if phonenumbers.is_possible_number(x) else "N"
        valid = "V" if phonenumbers.is_valid_number(x) else "N"
        country = x.country_code
        region = region_code_for_country_code(country)
        return("{} | {} | {} | {}".format(country, region, possible, valid))
    except:
        return("- | - | N | N")

submissions['phone_check'] = submissions['phone2'].apply(phone_check)

In [72]:
strange_phones = submissions[submissions['phone_check'] != "7 | RU | P | V"][
    ['student_id', 'airtable_id', 'submission_id', 'student_email', 'student_name', 'student_surname', 'city',
     'student_phone', 'phone2', 'phone_check']]

colnames = {
    'student_email': 'email',
    'student_name': 'name',
    'student_surname': 'surname',
    'student_phone': 'stored_phone',
    'phone2': 'guessed_phone'
}

strange_phones.rename(index=str, columns=colnames, inplace=True)

In [73]:
check = strange_phones['phone_check'].str.split('|', 3, expand=True).rename(
    columns={0:'code', 1:'region', 2:'possible', 3:'valid'})

strange_phones = strange_phones.join(check)
strange_phones

,student_id,airtable_id,submission_id,email,name,surname,city,stored_phone,guessed_phone,phone_check,code,region,possible,valid
145,A0718,NaN,P00182,illluminii@gmail.com,Марина,Рукавишникова,NaN,61490942371,+61490942371,61 | AU | P | V,61,AU,P,V
382,A0930,NaN,P00497,mashpanova@gmail.com,Маша,Панова,NaN,491732692492,+491732692492,49 | DE | P | V,49,DE,P,V
735,A1197,NaN,P00800,plekkatya@yandex.ru,Катерина,Плеханова,NaN,4915901344146,+4915901344146,49 | DE | P | V,49,DE,P,V
785,A1197,NaN,P00869,plekkatya@yandex.ru,Катерина,Плеханова,NaN,4915901344146,+4915901344146,49 | DE | P | V,49,DE,P,V
991,A0930,120,P01019,mashpanova@gmail.com,Маша,Панова,NaN,491732692492,+491732692492,49 | DE | P | V,49,DE,P,V
1104,A1513,256,P01189,vctria23m@gmail.com,Виктория,Матрюк,NaN,84591805940,+74591805940,7 | RU | P | N,7,RU,P,N
1256,A1637,466,P01319,xeniaaxelrod@gmail.com,Ксения,Аксельрод,NaN,99857764050,+99857764050,998 | UZ | N | N,998,UZ,N,N
1331,A1702,537,P01435,designpancha@gmail.com,Anastasia,Epanchintseva,NaN,14439601172,+14439601172,1 | US | P | V,1,US,P,V
1392,A1513,591,P01467,vctria23m@gmail.com,Виктория,Матрюк,NaN,84591805940,+74591805940,7 | RU | P | N,7,RU,P,N
1396,A1689,596,P01412,olizko1987@mail.ru,Полина,Олизько,NaN,447480269088,+447480269088,44 | GB | P | V,44,GB,P,V


In [66]:
from sugar import exportxlsx

exportxlsx(strange_phones, '', 'wrong_numbers')

In [74]:
strange_phones.to_csv("wrong_numbers.csv")